## GPT를 활용하여 식물의 상태 분석하기
- 식물의 기본 정보와 분석 데이터를 Chat GPT에 전송하여, 미리 정의된 출력 형식에 맞춘 응답을 받는다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
    - 1차 분석결과: 식물의 사진을 식물 질병 예측 모델(CNN)에 입력하여 식물의 질병을 예측한다.
    - 식물 환경 센서 데이터 : 라즈베리파이로 측정한 센서 데이터 값
    - 사용자가 입력한 식물 환경 정보 : 사용자가 직접 입력한 식물의 상태

  - output 형식 : json 형태
    - 구현 방식
      - 방식 1 : Text generation의 JSON mode 사용
      - 방식 2 : Function calling 사용

    

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


### 방식 1 : Text generation의 JSON mode

#### 처리 순서
1. 인풋 데이터 텍스트화
2. Chat GPT API 요청
  - 사전 준비 과정
    - 0. GPT API 초기 설정
    - 1. 어시스턴트 생성
    - 2. 어시스턴트 쓰레드 생성
    - 3. 어시스턴트 메시지 추가
    - 4. 어시스턴트 실행
3. 응답 파싱 및 가공
\* 인풋데이터는 원시데이터의 형태가 아닌 1차 가공된(날짜별 평균 및 최저 데이터) 데이터로 사전 작업해줘야한다.

##### 1. 인풋 데이터 텍스트화
- 인풋 데이터를 텍스트화 한다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
  - 해당 노트북에선 더미 데이터를 만들어 사용한다.

In [2]:
# 더미데이터

# 1차 분석 결과 데이터
predicted_disease = "healthy"

# 식물 환경 센서 데이터
env_data = {
  "2024-06-19": {
    "highest temperature": 41,
    "lowest temperature": 31,
    "average temperature": 38,
    "average humidity": "62%",
    "average illuminance": "7500 lux"
  },
  "2024-06-20": {
    "highest temperature": 42,
    "lowest temperature": 32,
    "average temperature": 39,
    "average humidity": "63%",
    "average illuminance": "7600 lux"
  },
  "2024-06-21": {
    "highest temperature": 43,
    "lowest temperature": 33,
    "average temperature": 40,
    "average humidity": "64%",
    "average illuminance": "7700 lux"
  },
  "2024-06-22": {
    "highest temperature": 44,
    "lowest temperature": 34,
    "average temperature": 41,
    "average humidity": "65%",
    "average illuminance": "7800 lux"
  },
  "2024-06-23": {
    "highest temperature": 45,
    "lowest temperature": 35,
    "average temperature": 42,
    "average humidity": "66%",
    "average illuminance": "7900 lux"
  },
  "2024-06-24": {
    "highest temperature": 46,
    "lowest temperature": 36,
    "average temperature": 43,
    "average humidity": "67%",
    "average illuminance": "8000 lux"
  },
  "2024-06-25": {
    "highest temperature": 47,
    "lowest temperature": 37,
    "average temperature": 44,
    "average humidity": "68%",
    "average illuminance": "8100 lux"
  },
  "2024-06-26": {
    "highest temperature": 48,
    "lowest temperature": 38,
    "average temperature": 45,
    "average humidity": "69%",
    "average illuminance": "8200 lux"
  },
  "2024-06-27": {
    "highest temperature": 49,
    "lowest temperature": 39,
    "average temperature": 46,
    "average humidity": "70%",
    "average illuminance": "8300 lux"
  },
  "2024-06-28": {
    "highest temperature": 50,
    "lowest temperature": 40,
    "average temperature": 47,
    "average humidity": "71%",
    "average illuminance": "8400 lux"
  }
}

# 사용자가 입력한 식물 환경 정보
user_input = {
  "Watering per day": "500ml",
  "wind hours per day": 2,
  "lighting hours per day": 3
}

In [6]:
def data_to_text(predicted_disease, env_data, user_input):
  # "다음 정보를 활용하여 식물을 분석해줘"
  result = "Analyze your plant using the following information.\n"

  # 1차 분석 결과 데이터
  result += f"\n1. Predicted disease: {predicted_disease}\n"

  # 식물 환경 센서 데이터
  result += f"\n2. Environment sensor data: \n"
  for date, sensor_data in env_data.items():
    result += f"Date: {date}\n"
    for key, value in sensor_data.items():
      result += f"- {key}: {value}\n"

  # 사용자가 입력한 식물 환경 정보
  result += f"\n3. User input: \n"
  for key, value in user_input.items():
    result += f"- {key}: {value}\n"

  return result

In [16]:
query=data_to_text(predicted_disease, env_data, user_input)

##### 2. Chat GPT API 요청
- 처리 과정
    - 0. GPT API 초기 설정
    - 1. 어시스턴트 생성
    - 2. 어시스턴트 쓰레드 생성
    - 3. 어시스턴트 메시지 추가
    - 4. 어시스턴트 실행

###### 0. GPT API 초기 설정
- API 키 불러오기
- Open AI 객체 생성

In [10]:
# env파일 로드
from dotenv import load_dotenv

dotenv_path = '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/env/gpt.env'
# verbose는 함수 작동시 추가정보 제공의 여부이다.
load_dotenv(dotenv_path, verbose=True)

True

In [11]:
import os
from openai import OpenAI

api_key = os.getenv('GPT_API_KEY')
organization = os.getenv('ORG_ID')
# project_id = os.getenv('PROJECT_ID')

client = OpenAI(
  organization=organization,
  api_key=api_key,
  # project = project_id
)

###### 1. 어시스턴트 생성
- JSON mode 설정
- 출력 포멧 설정

\* 어시스턴트에서는 최대 토큰을 제한할수없다 -> 실행단에서 제어

In [12]:
assistant = client.beta.assistants.create(
  name="이종현",
  instructions="you are an plant expert. Analyze the condition of the plant based on the entered plant information",
  model="gpt-3.5-turbo",
  response_format = {"type": "json_object"}
)

In [13]:
# 생성된 어이스턴트 목록 보기
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_XSDaQ1v6BoamEc2JvxZz7h5u', created_at=1718936817, description=None, instructions='you are an plant expert. Analyze the condition of the plant based on the entered plant information', metadata={}, model='gpt-3.5-turbo', name='이종현', object='assistant', tools=[], response_format=AssistantResponseFormat(type='json_object'), temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0), Assistant(id='asst_0HMzMgYqwzuNMQLVN1fWpV9p', created_at=1717915116, description=None, instructions='', metadata={}, model='gpt-4o', name='', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0), Assistant(id='asst_753aAlCG5EXiN3XSxb2f980y', created_at=1717915075, description=None, instructions='', metadata={}, model='gpt-3.5-turbo', name='json_response', object='assistant', tools=[], response_format=AssistantResponseFormat(type='json_object'), temperatu

###### 2. 어시스턴트 쓰레드 생성
- 쓰레드 생성 시 기본 메시지를 추가하여 반환한 JSON형식을 미리 정의해준다.

In [15]:
message_thread = client.beta.threads.create(
  messages=[
    {
      "role": "assistant",
      "content": "Are you a plant expert. Response in json format with Answer 'Crop condition',\
      'Current Status','Improvement plan'."
    },
    {
      "role": "assistant",
      "content": "'Crop conditions' range from 0 to 5. 0 means the plant is healthy and 5 means\
       the plant is in very critical condition."
    },
    {
      "role": "assistant",
      "content": "'Current Statu's expresses the current crop status in string form.."
    },
    {
      "role": "assistant",
      "content": "'improvement plan' expresses the measures necessary for crops to grow healthily in string form."
    },
  ]
)

print(message_thread)

Thread(id='thread_1kz1iJiD6HlfUQkX6IjcWfNb', created_at=1718938470, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


3. 어시스턴트 메시지 추가
- 생성했던 질의문을 쓰레드에 추가한다

In [18]:
thread_message = client.beta.threads.messages.create(
  "thread_1kz1iJiD6HlfUQkX6IjcWfNb", # 쓰레드 id
  role="user",
  content=query,
)

Message(id='msg_xdS0cQ7AHDQH4N5gvsM1macq', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Analyze your plant using the following information.\n\n1. Predicted disease: healthy\n\n2. Environment sensor data: \nDate: 2024-06-19\n- highest temperature: 41\n- lowest temperature: 31\n- average temperature: 38\n- average humidity: 62%\n- average illuminance: 7500 lux\nDate: 2024-06-20\n- highest temperature: 42\n- lowest temperature: 32\n- average temperature: 39\n- average humidity: 63%\n- average illuminance: 7600 lux\nDate: 2024-06-21\n- highest temperature: 43\n- lowest temperature: 33\n- average temperature: 40\n- average humidity: 64%\n- average illuminance: 7700 lux\nDate: 2024-06-22\n- highest temperature: 44\n- lowest temperature: 34\n- average temperature: 41\n- average humidity: 65%\n- average illuminance: 7800 lux\nDate: 2024-06-23\n- highest temperature: 45\n- lowest temperature: 35\n- average temperature: 42\n- av

4. 어시스턴트 실행
- 어시스턴트와 쓰레드를 사용하여 GPT에서 답변을 응답 받는다.

- 어시스턴트 실행에선 추가적으로 instructions을 설정할 수 있다.
  - 보통 사용자의 이름을 지정하거나 추가정보를 기입한다.

In [21]:
run = client.beta.threads.runs.create(
  thread_id="thread_1kz1iJiD6HlfUQkX6IjcWfNb",
  assistant_id="asst_XSDaQ1v6BoamEc2JvxZz7h5u",
  instructions="The user's name is 'LEE JONGHYEON'.",
  # max_prompt_tokens= 600 # 옵션
)

print(run)

Run(id='run_Lv6dg7kKTbeSFZGSUygQGY0p', assistant_id='asst_XSDaQ1v6BoamEc2JvxZz7h5u', cancelled_at=None, completed_at=None, created_at=1718939045, expires_at=1718939645, failed_at=None, incomplete_details=None, instructions='you are an plant expert. Analyze the condition of the plant based on the entered plant information', last_error=None, max_completion_tokens=None, max_prompt_tokens=600, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format=AssistantResponseFormat(type='json_object'), started_at=None, status='queued', thread_id='thread_1kz1iJiD6HlfUQkX6IjcWfNb', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [40]:
import json

message = client.beta.threads.messages.list(
    thread_id="thread_1kz1iJiD6HlfUQkX6IjcWfNb",
    order="desc"
)

print(message.data)
answer = message.data[1].content[0].text.value

print(answer)

# JSON 문자열을 파이썬 객체로 변환
parsed_json = json.loads(answer)

# 파이썬 객체를 이쁘게 출력된 JSON 문자열로 변환
pretty_json = json.dumps(parsed_json, indent=4, ensure_ascii=False)

print(pretty_json)

[Message(id='msg_QPyvwKS37KLnPZ9URiyCpPiH', assistant_id='asst_XSDaQ1v6BoamEc2JvxZz7h5u', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{"plant_condition":"Based on the provided information, the plant seems to be in a healthy condition. The environmental sensor data shows relatively high temperatures ranging from 41 to 50 degrees Celsius, but this doesn\'t indicate a specific issue. The average humidity levels are within a reasonable range of 62% to 71%, which is suitable for plant growth. The illuminance levels are also consistently high, which is beneficial for the plant. The watering per day, wind hours per day, and lighting conditions are not mentioned in the user input, so further details on these factors would be needed for a more comprehensive analysis."}'), type='text')], created_at=1718939047, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Lv6dg7kKTbeSFZGSUygQGY0p'

JSONDecodeError: Expecting value: line 1 column 1 (char 0)